In [1]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
import seaborn as sns

In [2]:
#import original csv file from Rides Database
file="saferparks.csv"
pd.set_option('display.max_columns', None)
df = pd.read_csv(file, encoding='cp1252')
df.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,tradename_or_generic,manufacturer,num_injured,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes
0,1005813,6/12/2010,OH,Cleveland,F,Ohio Dept. of Agriculture,Sports or recreation facility,recreation,inflatable,Inflatable slide,inflatable slide,Scherba Industries / Inflatable Images,9.0,NaN,NaN,Inflatable slide tipped over while 7-9 patrons...,The man who was crushed by the device died 9 d...,https://ridesdatabase.org/accident_reports/OH-...,"Device tipped over, blew away, or collapsed",NaN,1.0,NaN,http://www.cleveland.com/metro/index.ssf/2012/...
1,1004032,6/12/2010,OH,Cleveland,P,United States Consumer Product Safety Commission,Sports or recreation facility,recreation,inflatable,Inflatable slide,inflatable slide,Scherba Industries / Inflatable Images,8.0,54.0,M,Two adult males were standing next to an infla...,1 - death from pulmonary embolism caused by bl...,https://ridesdatabase.org/accident_reports/CPS...,"Device tipped over, blew away, or collapsed",NaN,1.0,NaN,See also OH state record.
2,1007658,7/10/2010,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,water ride,Boat ride,boat ride,Arrow Dynamics,1.0,37.0,F,Patron stated she slipped and fell while stepp...,"Hip and low back pain, torn meniscus in knee",NaN,Load/Unload: scrape or stumble,NaN,NaN,NaN,NaN
3,1007098,7/10/2010,CA,Carlsbad,F,California Division of Occupational Safety and...,Water park,water park,float attraction,Lazy river,lazy river,California Pools,1.0,37.0,F,Patron claims strained back after her raft tip...,Strained back,NaN,"Fall: patron fell off inner tube, mat or board",NaN,NaN,NaN,NaN
4,1000094,7/29/2010,CO,Littleton,F,Colorado Division of Oil and Public Safety,Family entertainment center,recreation,challenge activity,Climbing wall,climbing wall,Spectrum Sports,1.0,NaN,M,"Two patrons climbed to the top of the wall, co...",Patron experienced a free fall that resulted i...,https://ridesdatabase.org/accident_reports/CO-...,Fall: patron fell from device (participatory),1.0,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8351 entries, 0 to 8350
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   acc_id                8351 non-null   int64  
 1   acc_date              8351 non-null   object 
 2   acc_state             8351 non-null   object 
 3   acc_city              8233 non-null   object 
 4   fix_port              8351 non-null   object 
 5   source                8351 non-null   object 
 6   bus_type              8351 non-null   object 
 7   industry_sector       8351 non-null   object 
 8   device_category       8351 non-null   object 
 9   device_type           8351 non-null   object 
 10  tradename_or_generic  8351 non-null   object 
 11  manufacturer          5041 non-null   object 
 12  num_injured           8349 non-null   float64
 13  age_youngest          7667 non-null   float64
 14  gender                7623 non-null   object 
 15  acc_desc             

In [ ]:
df["acc_date"] = pd.to_datetime(df["acc_date"])
df.info()

In [ ]:
df.head()

In [5]:
df.acc_date.value_counts()

8/16/2014     23
7/21/2012     22
7/11/2015     21
6/14/2015     20
7/19/2015     20
              ..
11/26/2015     1
11/20/2015     1
2/26/2013      1
2/27/2013      1
7/26/2017      1
Name: acc_date, Length: 1845, dtype: int64

In [4]:
#import kaggle clean csv file from Rides Database
file="safer-parks-accident-dataset.csv"
pd.set_option('display.max_columns', None)
df1 = pd.read_csv(file, encoding='cp1252')
df1.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,tradename_or_generic,manufacturer,num_injured,age_youngest,gender,acc_desc,injury_desc,report,category,mechanical,op_error,employee,notes,year
0,1007272,2009-04-05,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Track ride,dark ride,In-house,1,44.0,F,Patron's attorney stated that she had a right ...,Right vertegral artery tear,0,Illness or neurological symptoms,False,False,False,0,2009
1,918972,2009-02-05,FL,Naples,P,Florida Dept. of Agriculture,Carnival or rental,amusement ride,pendulum,Flying carpet ride,Avalanche,"Wisdom Industries, Ltd.",1,17.0,F,17-yo female was ejected from the ride mid-cyl...,Multiple broken ribs and bruised lung,https://saferparksdata.org/sites/default/files...,Fall: ejection/fall from ride,True,False,False,"This was the third ejection on this ride, unde...",2009
2,919123,2008-11-08,CA,Buena Park,F,California Division of Occupational Safety and...,Amusement park,amusement ride,coaster,Coaster - steel,Boomerang,Vekoma,1,19.0,M,Guest injured right hand while riding.,Injured hand. Treated at local hospital.,0,Unknown (not enough info),False,False,False,0,2008
3,919095,2008-11-01,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,spinning,Spinning cups/tubs,tea cups,In-house,1,51.0,F,"Guest stated when she sat down in the vehicle,...",Nosebleed. Treated at local hospital.,0,Illness or neurological symptoms,False,False,False,0,2008
4,919094,2008-10-29,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,spinning,Spinning cups/tubs,tea cups,In-house,1,38.0,F,Guest stated she felt lightheaded and experien...,"Dizziness, high blood pressure. Treated at lo...",0,Illness or neurological symptoms,False,False,False,0,2008


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14884 entries, 0 to 14883
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   acc_id                14884 non-null  int64  
 1   acc_date              14884 non-null  object 
 2   acc_state             14884 non-null  object 
 3   acc_city              14884 non-null  object 
 4   fix_port              14884 non-null  object 
 5   source                14884 non-null  object 
 6   bus_type              14884 non-null  object 
 7   industry_sector       14884 non-null  object 
 8   device_category       14884 non-null  object 
 9   device_type           14884 non-null  object 
 10  tradename_or_generic  14884 non-null  object 
 11  manufacturer          14884 non-null  object 
 12  num_injured           14884 non-null  int64  
 13  age_youngest          14884 non-null  float64
 14  gender                14884 non-null  object 
 15  acc_desc           

In [7]:
df1["acc_date"] = pd.to_datetime(df1["acc_date"])
df1["year"] = df1["year"].astype(int)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14884 entries, 0 to 14883
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   acc_id                14884 non-null  int64         
 1   acc_date              14884 non-null  datetime64[ns]
 2   acc_state             14884 non-null  object        
 3   acc_city              14884 non-null  object        
 4   fix_port              14884 non-null  object        
 5   source                14884 non-null  object        
 6   bus_type              14884 non-null  object        
 7   industry_sector       14884 non-null  object        
 8   device_category       14884 non-null  object        
 9   device_type           14884 non-null  object        
 10  tradename_or_generic  14884 non-null  object        
 11  manufacturer          14884 non-null  object        
 12  num_injured           14884 non-null  int64         
 13  age_youngest    

In [8]:
df1.year.value_counts()

2001    2217
2002    1864
2000    1818
2005    1409
2003    1325
2004    1255
2006    1202
1999    1015
2007     934
1997     318
1998     263
1995     226
1996     224
2008     204
1994     194
1993     134
1992      92
1990      84
1989      40
1988      31
1991      31
2009       2
1987       1
1986       1
Name: year, dtype: int64

In [ ]:
cleandf=df1.sample(n=1000)
cleandf

In [ ]:
cleandf.info()

In [ ]:
# df.dropna(axis=0, subset="acc_city", inplace=True)

In [ ]:
# df["address"] =df.acc_city + ", " + df.acc_state
# df.head()

In [ ]:
# df.rename(columns = {'Runtime':'Runtime_mins'}, inplace = True)
# df.head()